In [1]:
try: 
    import sys
    import numpy as np
    import tifffile as tf
    import os
    from osgeo import gdal, osr
    import earthpy as et
    import earthpy.spatial as es
    import math
    import imagecodecs
    #print('Import OK.')
    
except ImportError as IE:
    print('Import not OK - ', IE )
    sys.exit('exit - bad import')

In [2]:
#https://gist.github.com/jkatagi/a1207eee32463efd06fb57676dcf86c8
def fGeoref(input_array, src_dataset_path, output_path):
        cols = input_array.shape[1]
        rows = input_array.shape[0]
        
        dataset = gdal.Open(src_dataset_path, gdal.GA_ReadOnly)
        originX, pixelWidth, b, originY, d, pixelHeight = dataset.GetGeoTransform() 

        driver = gdal.GetDriverByName('GTiff')
        band_num = 1
        GDT_dtype = gdal.GDT_Float32
        outRaster = driver.Create(output_path, cols, rows, band_num, GDT_dtype)
        outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
        outband = outRaster.GetRasterBand(band_num)
        outband.WriteArray(input_array)
        prj=dataset.GetProjection()
        outRasterSRS = osr.SpatialReference(wkt=prj)
        outRaster.SetProjection(outRasterSRS.ExportToWkt())
        
        return True


In [3]:
def fndvi(red_path, nir_path, out_folder, name = "_NDVI"):

    red= tf.imread(red_path)
    nir = tf.imread(nir_path)

    zero_exception = np.seterr(all = "ignore")
    red_array = np.array(red).astype(np.float32)
    nir_array = np.array(nir).astype(np.float32)
    
    #tf.imsave(os.path.join(OUTPUT, "red.TIF"), (red_array+nir_array))
    #tf.imsave(os.path.join(OUTPUT, "nir.TIF"), (nir_array-red_array))
    
    result_ndvi = np.divide((nir_array-red_array),(red_array+nir_array))
      
    result_ndvi_path = os.path.join(out_folder, name + ".TIF") 

    #tf.imsave(result_ndvi_path, result_ndvi)
    fGeoref(result_ndvi, red_path, result_ndvi_path)
    
    return result_ndvi_path


In [4]:
def fTOA_Refl(band_path, Ap, Mp, out_folder, name = "_TOA_Ref"):   
    
    band= tf.imread(band_path)
    
    result_toa = Mp * band + Ap
    result_toa_path = os.path.join(out_folder, name + ".TIF")
    
    #tf.imsave(result_toa_path , result_toa)
    
    fGeoref(result_toa, band_path, result_toa_path)
    
    return result_toa_path

In [5]:
def fTOA_Rad(thermo_path, ML, AL, out_folder, name = "TOA_Rad"):   
        
    thermo= tf.imread(thermo_path)
    
    result_toa_rad = (ML  * thermo) + AL
    result_toa_rad_path = os.path.join(out_folder, name + ".TIF")
    
    #tf.imsave(result_toa_rad_path , result_toa_rad)
    
    fGeoref(result_toa_rad, thermo_path, result_toa_rad_path)
    
    return result_toa_rad_path

In [6]:
#vyzarena teplota bez korekce pro emisivitu

def fBT(input_path_toa, K1, K2, out_folder, name = "BT"):
    
    #K1 = 774.8853 #Kelvin constant
    #K2 = 1321.0789 #Kelvin constant
    
    toa_array = np.array(tf.imread(input_path_toa))
    
    zero_exception = np.seterr(all = "ignore")
        
    result_BT = K2/np.log(((K1/toa_array+1))) - 273.15
    #result_BT[result_BT == np.inf] = 0    
    
    result_bt_path = os.path.join(out_folder, name + ".TIF") 
    
    fGeoref(result_BT, input_path_toa, result_bt_path)
    
    #tf.imsave(result_bt_path, result_BT)
    
    return result_bt_path


In [7]:
def fVc(ndvi, out_folder, name = "Vc"):
    ndvi_array = np.array(tf.imread(ndvi))
    result_Vc = np.divide(np.power(ndvi_array,2),0.3)

    result_Vc_path = os.path.join(out_folder, name +".TIF")

    #tf.imsave(result_Vc_path, result_Vc)

    fGeoref(result_Vc, ndvi, result_Vc_path)

    return result_Vc_path

In [8]:
def fRadLongIn(emis_atmo, temp_air_kel, name = "RadTLongIn"):
    
    result_long_in = emis_atmo * 5.6703 * 10.0 ** (-8.0) * temp_air_kel ** 4
    return result_long_in

In [9]:
def fEmis(vc, ndvi, red, out_folder, name = "Emis"):
        
    ndvi_array = np.array(tf.imread(ndvi))
    vc_array = np.array(tf.imread(vc))
    red = np.array(tf.imread(red))

    result_e = (0.004 * vc_array) + 0.986
    #print(result_e)
    result_e = np.where(ndvi_array < 0.2, 1 - red, result_e) 
    result_e = np.where(ndvi_array > 0.5, 0.99, result_e)              
    result_e[result_e > 1] = 0.99                           
    result_e[result_e < 0.8] = 0.8
    #print(result_e)

    result_e_path = os.path.join(out_folder, name +".TIF")

    fGeoref(result_e, vc, result_e_path)

    #tf.imsave(result_e_path, result_e)
    return result_e_path

In [10]:
def fLST(bt, surf_emis, out_folder, name = "LST"):
        bt_array = np.array(tf.imread(bt))
        e_array = np.array(tf.imread(surf_emis))
        
        result_LST = (bt_array / (1 + ((0.0015 * bt_array)/1.4488) * np.log(e_array)))
        
        result_LST_path = os.path.join(out_folder, name + ".TIF")
        
        fGeoref(result_LST, bt, result_LST_path)
        
        #tf.imsave(result_LST_path, result_LST)
        return result_LST_path

In [11]:
# e = surface emis

def fRadLongOut(lst, surf_emis, out_folder, name = "RadLongOut"):
    lst_array = np.array(tf.imread(lst))
    e_array = np.array(tf.imread(surf_emis))
    
    result_long_out = e_array * 5.67 * 10 ** (-8.0) * (lst_array + 273.15)**4
        
    long_out_path = os.path.join(out_folder, name +".TIF")
    
    fGeoref(result_long_out, lst, long_out_path)
    
    #tf.imsave(long_out_path, result_long_out)
    #print(result_long_out)
    return long_out_path

In [12]:
def fAlbedo(toa_band_1, toa_band_3, toa_band_4, toa_band_5, toa_band_7, out_folder, name = 'albedo'):
    band1_array = np.array(tf.imread(toa_band_1))
    band3_array = np.array(tf.imread(toa_band_3))
    band4_array = np.array(tf.imread(toa_band_4))
    band5_array = np.array(tf.imread(toa_band_5))
    band7_array = np.array(tf.imread(toa_band_7))
    
    result_albedo = ((0.356 * band1_array) + (0.130 * band3_array) + (0.373 * band4_array) + (0.085 * band5_array) + (0.072 * band7_array))-0.0018/1.016
    result_albedo_path = os.path.join(out_folder, name + ".TIF")
    
    fGeoref(result_albedo, toa_band_4, result_albedo_path)
      
    #tf.imsave(result_albedo_path, result_albedo)
    
    return result_albedo_path

In [13]:
def fRadShortOut(RadShIn, albedo, out_folder, name ="RadShortOut"):
    
    albedo_array = np.array(tf.imread(albedo))
    
    result_RadShortOut = albedo_array * RadShIn
    
    result_short_out_path = os.path.join(out_folder, name +".TIF")
    
    fGeoref(result_RadShortOut, albedo, result_short_out_path)
    
    #tf.imsave(result_short_out_path, result_RadShortOut)
    
    return result_short_out_path

In [14]:
def fRadLongIn(emis_atmo, air_temp_kel, name = "RadTLongIn"):
    
    result_long_in = emis_atmo * 5.6703 * 10.0 ** (-8.0) * air_temp_kel **4
    
    return result_long_in

In [15]:
def fTotalRad(RadShortIn, RadShortOut, RadLongIn, RadLongOut, out_folder, name = "TotalRad"):
    
    RadShortOut_array = np.array(tf.imread(RadShortOut))
    RadLongOut_array = np.array(tf.imread(RadLongOut))
    
    result_TotalRad = (RadShortIn - RadShortOut_array) + (RadLongIn - RadLongOut_array)
    
    result_TotalRad_path = (os.path.join(out_folder, name + ".TIF"))
    
    fGeoref(result_TotalRad, RadLongOut, result_TotalRad_path)
    
    #tf.imsave(result_TotalRad_path, result_TotalRad)
    
    return result_TotalRad_path

In [16]:
def fGroundHeatFl(albedo, ndvi, lst, TotalRad, out_folder, name = "GroundHeatFlux"):
    albedo_array = np.array(tf.imread(albedo))
    ndvi_array = np.array(tf.imread(ndvi))
    lst_array = np.array(tf.imread(lst))
    TotalRad_array = np.array(tf.imread(TotalRad))
    
    result_ghf_a = (0.0038 * albedo_array) + (0.0074 * albedo_array)
    result_ghf_b = (1 - 0.98) * ndvi_array
    result_ghf_c = (np.power(result_ghf_a,2) * np.power(result_ghf_b,4)) * TotalRad_array
    result_ghf_d = albedo_array * result_ghf_c
    result_ghf = np.divide(lst_array, result_ghf_d)
 
    result_ghf = lst_array/albedo_array * (0.0038 * albedo_array + 0.0074 * albedo_array**2) * (1 - 0.98 * ndvi_array**4) * TotalRad_array
    result_ghf_path = (os.path.join(out_folder, name + ".TIF"))
    
    fGeoref(result_ghf, ndvi, result_ghf_path)
    #tf.imsave(result_ghf_path, result_ghf)
    
    return result_ghf_path

In [17]:
def fEvapoFraction(lst, air_temp, out_folder, name = 'EvapoFraction'):
    lst_array = np.array(tf.imread(lst))
    
    lst_median = ndimage.median_filter(lst_array, 5)
    lst_median = lst_median[~np.isnan(lst_median)]
    Tmax = np.nanmax(lst_array)
    evapo_frac = np.divide(Tmax - lst_array, Tmax - air_temp)
    
    evapo_frac_path = (os.path.join(out_folder, name + ".TIF"))
    fGeoref(evapo_frac, lst, evapo_frac_path)
    
    #tf.imsave(evapo_frac_path, evapo_frac)
    
    return evapo_frac_path

In [18]:
def fAvailableEvapo(TotalRad, ground_heat_flux, out_folder, name = 'AvailableEvapo'):
    TotalRad_array = np.array(tf.imread(TotalRad))
    ground_heat_flux_array= np.array(tf.imread(ground_heat_flux))
    
    avialable_evapo = TotalRad_array - ground_heat_flux_array #available energy for evaporation

    avialable_evapo_path = (os.path.join(out_folder,  name + ".TIF"))  
    fGeoref(avialable_evapo, TotalRad, avialable_evapo_path)
    
    #tf.imsave(avialable_evapo_path, avialable_evapo)
    
    return avialable_evapo_path

In [19]:
def fLatentHeatFlux(evapo_frac, available_evapo, out_folder, name = "LatentHeatFlux" ):    
    evapo_frac_array = np.array(tf.imread(evapo_frac))
    avialable_evapo_array = np.array(tf.imread(available_evapo))
    
    latent_heat_flux = evapo_frac_array * avialable_evapo_array
    
    latent_heat_flux_path = (os.path.join(out_folder,  name + ".TIF"))
    fGeoref(latent_heat_flux, evapo_frac, latent_heat_flux_path)
    
    #tf.imsave(latent_heat_flux_path, latent_heat_flux)
    
    return latent_heat_flux_path

In [20]:
def fSensibHeatFl(latent_heat_flux, available_evapo, out_folder, name = "SensibleHeatFlux" ):
    latent_heat_flux_array = np.array(tf.imread(latent_heat_flux))
    available_evapo_array = np.array(tf.imread(available_evapo))

    result_sensible_heat_flux = available_evapo_array - latent_heat_flux_array

    result_sensible_heat_flux_path = (os.path.join(out_folder, name + ".TIF"))

    
    fGeoref(result_sensible_heat_flux, available_evapo, result_sensible_heat_flux_path)
   
    
    #tf.imsave(result_sensible_heat_flux_path, result_sensible_heat_flux)
    
    
    return result_sensible_heat_flux_path
  

In [21]:
def fSaturVapPress(air_temp_c):
    SatVapPress = 4098*(0.610*math.exp((17.27*air_temp_c)/(air_temp_c+237.3))/(air_temp_c+237.3)**2)
    return SatVapPress

In [22]:
def fPsychCons(AtmPress, latent_heat, cp, epsilon):
    Y = (cp * AtmPress) / (epsilon * latent_heat)
    return Y

In [23]:
def fET0(ea, es_v, WindSp, SaturVapPress, GroundHeatFlux, psych, TotalRad, TEMP_K, out_folder, name= "ET0"):
    ground_heat_flux = np.array(tf.imread(GroundHeatFlux)) 
    total_rad = np.array(tf.imread(TotalRad))

    
    a = 0.408 * SaturVapPress * (ground_heat_flux - total_rad)
    b = psych * (900 / TEMP_K) * (WindSp * (es_v - ea))
    c = SaturVapPress + psych * (1 + (0.34 * WindSp))
    ET0 = np.divide((a + b),c)
    ET0_path = (os.path.join(out_folder, name + '.TIF'))     
    
    fGeoref(ET0, TotalRad, ET0_path)
    #tf.imsave(ET0_path, ET0)

    return ET0_path


In [24]:
####################PŘEMĚNIT NP MAX!!!!##################################
#np.max(ET0)

def fETI(Kc, ET0, out_folder, name = 'ETI'):
    ET0 = np.array(tf.imread(ET0))
    ETI = (Kc * ET0)/np.max(100)
    ETI_path = (os.path.join(out_folder, name + ".TIF"))

    fGeoref(ETI, ET0, ETI_path)
    #tf.imsave(ETI_path, ETI)

    return ETI_path


In [25]:
def fshade(dem, azimuth = 315, altitude=45, name = "SHADE"):
    elevation = tf.imread(dem)
    #Divided by 255 to standardize from hillshade scale (0-255) to 0-1 scale
    result_shade = es.hillshade(elevation, azimuth, altitude)/255
    
    result_shade_path = os.path.join(out_folder, 'Shade' + ".TIF")
    
    #fGeoref(result_shade, FILE_BAND_4, result_shade_path)
    
    tf.imsave(result_shade_path, result_shade)
    return result_shade_path

In [26]:
def fCCi(albedo, eti, out_folder, name = 'CCi'):
    #shade = np.array(tf.imread(shade))
    albedo_array = np.array(tf.imread(albedo))
    ETI_array = np.array(tf.imread(eti))
    
    #CCi = (0.6 * shade) + (0.2 * albedo) + (0.2 * ETI)
    #CCi = (0.2 *shade_array) + (0.2 * ETI_array)
    CCi_path = (os.path.join(out_folder, name + ".TIF"))
    #fGeoref(CCi, albedo, CCi_path)

    #tf.imsave(CCi_path, CCi)
    return CCi_path